In [1]:
import torch
from torch import nn

# 인공신경망 만들기

In [3]:
x = torch.tensor([1.])
model = nn.Linear(1,1) # 입력 node 한 개, 출력 node 한 개인 layer 만든다

print(model.weight) # 만들면서 initilaize 해준다
print(model.bias)
print()

y = model(x)
print(y)

Parameter containing:
tensor([[0.5586]], requires_grad=True)
Parameter containing:
tensor([-0.9019], requires_grad=True)

tensor([-0.3434], grad_fn=<AddBackward0>)


실제 수식으로 비교해보자

In [4]:
y = x @ model.weight + model.bias
print(y)

tensor([-0.3434], grad_fn=<AddBackward0>)


동일한 값인 것을 볼 수 있다

In [7]:
x = torch.tensor([1.])

fc1 = nn.Linear(1,3) # input : 1, output : 3인 node로 구성 = fully-connected
fc2 = nn.Linear(3,1)

print("----- fc1 -----")
print(fc1.weight)
print(fc1.bias)
print("----- fc2 -----")
print(fc2.weight)
print(fc2.bias)
print()

x1 = fc1(x)
x2 = fc2(x1)
print(x1)
print(x2)

----- fc1 -----
Parameter containing:
tensor([[-0.3988],
        [ 0.1842],
        [-0.1550]], requires_grad=True)
Parameter containing:
tensor([-0.1147,  0.6470,  0.3014], requires_grad=True)
----- fc2 -----
Parameter containing:
tensor([[-0.1379,  0.2214,  0.0785]], requires_grad=True)
Parameter containing:
tensor([0.3519], requires_grad=True)

tensor([-0.5135,  0.8312,  0.1464], grad_fn=<AddBackward0>)
tensor([0.6183], grad_fn=<AddBackward0>)


마찬가지로 수식으로!

In [9]:
y = (x@fc1.weight + fc1.bias)@fc2.weight + fc2.bias
## (1x1 @ 1x3 + 1x3)@ 3x1 + 1x1) ?

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1 and 3x1)

에러가 난다 !

<img src="image/nn_linear.png" width="600">

nn.Linear 안에서 Transpose가 되어있기 때문!

왜 T 되어있느냐?
- shape는 개x채x행x열 로 이루어져있다
  - x = torch.tensor([1., 2., 3.]) => 1x3 = 1행 3열 = 1개 3채널 로도 볼 수 있다
- nn.linear은 개x채x행x열 에서 input으로 "채널"이 들어오길 기대한다
  - nn.linear가 채널 수를 input으로 받는 이유
    - 데이터의 개수는 여러번 반복해서 들어가는 것이지, 노드의 수를 결정하는 것은 채널의 수이기 때문
  - x가 1개 3채널이므로, nn.linear(3, ?) 넣어주듯이 채널이 들어오길 기대한다
- nn.linear 같은 경우, weight가 개x채의 형태를 띄게끔 한다
- 여기서 입력된 채널 수와 동일한 채널을 가진다
  - nn.linear(3, 5) => 입력된 3 채널 (input)
  - 이와 동일한 채널 수를 먼저 weight가 구성된다 > ?x3 (개x채)
  - 그리고 나머지 5개의 노드 수를 맞추기 위해서 > 5x3 (개x채) 가 된다
- 이를 위해서 Transpose가 들어가 있는 것 !

In [12]:
y = (x@fc1.weight.T + fc1.bias)@fc2.weight.T + fc2.bias
## (1x1 @ 1x3 + 1x3)@ 3x1 + 1x1)
print(y)
print()

print("----- shape -----")
print(f"x.shape : {x.shape}")
print(f"fc1.weight : {fc1.weight.shape}")
print(f"fc1.bias : {fc1.bias.shape}")
print(f"fc2.weight : {fc2.weight.shape}")
print(f"fc2.bias : {fc2.bias.shape}")
print(f"y.shape : {y.shape}")

tensor([0.6183], grad_fn=<AddBackward0>)

----- shape -----
x.shape : torch.Size([1])
fc1.weight : torch.Size([3, 1])
fc1.bias : torch.Size([3])
fc2.weight : torch.Size([1, 3])
fc2.bias : torch.Size([1])
y.shape : torch.Size([1])


In [13]:
x = torch.tensor([1.])

fc1 = nn.Linear(1,3)
fc2 = nn.Linear(3,1)

model = nn.Sequential(fc1, fc2) # layer 붙이기!
model(x)

tensor([0.0465], grad_fn=<AddBackward0>)

In [19]:
model = nn.Sequential(nn.Linear(2, 5),
                      nn.Linear(5, 10),
                      nn.Linear(10, 3))

x = torch.randn(2)
print(model(x)) # 1x2 2x5 5x10 10x3 = 1x3

x = torch.randn(5,2) # 5개 2채널 (개x채)
print(model(x)) # 채널 수가 들어가 나오므로, 5x3

x = torch.randn(3, 4, 5, 2, 3, 5,2) # 앞 개수가 계속 추가되어도 가능하다!
print(model(x).shape)

tensor([ 0.0241, -0.2140,  0.3608], grad_fn=<AddBackward0>)
tensor([[ 0.1444, -0.0622,  0.1991],
        [ 0.0230,  0.0630,  0.4521],
        [-0.0132, -0.0383,  0.4829],
        [ 0.2429, -0.0606,  0.0270],
        [-0.0350,  0.2577,  0.6164]], grad_fn=<AddmmBackward0>)
torch.Size([3, 4, 5, 2, 3, 5, 3])


## Model class 만들기

기본틀!

In [26]:
class Mymodel(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    pass
    return x

In [27]:
class Mymodel(nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = nn.Linear(3,5)
    self.fc2 = nn.Linear(5,10)
    self.fc3 = nn.Linear(10,2)
    self.sg1 = nn.Sigmoid()
    self.sg2 = nn.Sigmoid()
    self.sg3 = nn.Sigmoid()

  def forward(self, x):
    x = self.fc1(x)
    x = self.sg1(x)
    x = self.fc2(x)
    x = self.sg2(x)
    x = self.fc3(x)
    x = self.sg3(x)
    return x

In [28]:
model = Mymodel()
x = torch.randn(5, 3)
model(x)

tensor([[0.5499, 0.4168],
        [0.5472, 0.4179],
        [0.5491, 0.4182],
        [0.5497, 0.4193],
        [0.5510, 0.4172]], grad_fn=<SigmoidBackward0>)

In [29]:
# 모델 확인
print(model)
print(model.fc1)
print(model.fc1.weight)

Mymodel(
  (fc1): Linear(in_features=3, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
  (sg1): Sigmoid()
  (sg2): Sigmoid()
  (sg3): Sigmoid()
)
Linear(in_features=3, out_features=5, bias=True)
Parameter containing:
tensor([[-0.5597, -0.4522, -0.3611],
        [-0.0340, -0.2003, -0.0444],
        [-0.3687, -0.5011,  0.0886],
        [ 0.4488,  0.0336,  0.3306],
        [-0.0227,  0.3573,  0.4370]], requires_grad=True)


In [32]:
# Sequential 사용
class Mymodel2(nn.Module):
  def __init__(self):
    super().__init__()

    # self.fc1 = nn.Linear(3,5)
    # self.fc2 = nn.Linear(5,10)
    # self.fc3 = nn.Linear(10,2)
    # self.sg1 = nn.Sigmoid()
    # self.sg2 = nn.Sigmoid()
    # self.sg3 = nn.Sigmoid()
    self.linear = nn.Sequential(
      nn.Linear(3,5),
      nn.Sigmoid(),
      nn.Linear(5,10),
      nn.Sigmoid(),
      nn.Linear(10,2),
      nn.Sigmoid(),
    )

  def forward(self, x):
    x = self.linear(x)
    return x

In [33]:
model = Mymodel2()
x = torch.randn(5, 3)
model(x)

tensor([[0.4159, 0.5959],
        [0.4160, 0.5965],
        [0.4129, 0.5981],
        [0.4150, 0.5967],
        [0.4158, 0.5975]], grad_fn=<SigmoidBackward0>)

In [35]:
# 모델 확인
print(model)
print(model.linear[0].weight) # 인덱스로 fc1 접근
print(model.linear[1])

Mymodel2(
  (linear): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=5, out_features=10, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=10, out_features=2, bias=True)
    (5): Sigmoid()
  )
)
Parameter containing:
tensor([[-0.1965, -0.2415, -0.2549],
        [ 0.2851,  0.1767,  0.0094],
        [ 0.5686,  0.0639, -0.1442],
        [ 0.4942,  0.2143, -0.1192],
        [ 0.3570,  0.0703, -0.3899]], requires_grad=True)
Sigmoid()


In [37]:
# 안에 있는 모든 파라미터 확인
list(model.parameters()) # weight, bias 순서로

[Parameter containing:
 tensor([[-0.1965, -0.2415, -0.2549],
         [ 0.2851,  0.1767,  0.0094],
         [ 0.5686,  0.0639, -0.1442],
         [ 0.4942,  0.2143, -0.1192],
         [ 0.3570,  0.0703, -0.3899]], requires_grad=True),
 Parameter containing:
 tensor([0.2856, 0.2087, 0.4699, 0.3086, 0.4645], requires_grad=True),
 Parameter containing:
 tensor([[-0.3447, -0.1969, -0.3284, -0.3139,  0.0427],
         [-0.1375, -0.1848, -0.2784, -0.4447,  0.1528],
         [-0.0649,  0.0055, -0.4182,  0.2248, -0.2208],
         [ 0.1901,  0.1547, -0.3315,  0.2063,  0.0673],
         [-0.2925, -0.4321,  0.3483,  0.1526, -0.0471],
         [-0.2469,  0.0486,  0.0498, -0.4134, -0.2500],
         [ 0.2269, -0.0231,  0.3452,  0.3079,  0.3826],
         [ 0.1112, -0.1615,  0.4073, -0.4265,  0.2303],
         [-0.1285,  0.0678,  0.1108,  0.0757, -0.1835],
         [-0.1418,  0.2732,  0.2416,  0.4188,  0.1994]], requires_grad=True),
 Parameter containing:
 tensor([-0.0014,  0.1581,  0.0352, -0.4125

In [38]:
# 파라미터의 원소 개수 확인
[i.numel() for i in model.parameters()] # weight, bias 순서로

[15, 5, 50, 10, 20, 2]

# weight initialization

In [39]:
# weight initialization을 직접할 수 있다 !
# He initialization 이 공식 문서랑 다르다? => paper에 맞게 구현됐고 torch 공식 문서만 틀림 ???

Fin=5000
Fout=1000
w = torch.zeros(141, Fin)
nn.init.kaiming_uniform_(w, mode='fan_in', nonlinearity='relu') # forward pass에서 값의 범위를 유지시켜주기 위함
print(w.std())
print(torch.sqrt(torch.tensor(2/Fin)))
w = torch.zeros(Fout, 212)
nn.init.kaiming_uniform_(w, mode='fan_out', nonlinearity='relu') # backward pass에서 값의 범위를 유지시켜주기 위함
print(w.std())
print(torch.sqrt(torch.tensor(2/Fout)))

# CNN?
N=32
C=64
H=6
W=10
w = torch.zeros(N,C,H,W)
nn.init.kaiming_uniform_(w, mode='fan_in', nonlinearity='relu')
print(w.std())
print(torch.sqrt(torch.tensor(2/(C*H*W))))

w = torch.zeros(N,C,H,W)
nn.init.kaiming_uniform_(w, mode='fan_out', nonlinearity='relu')
print(w.std())
print(torch.sqrt(torch.tensor(2/(N*H*W))))

In [ ]:
1:19:16